# This notebook will walk you through creating the Solr Database and the MariaDB  table containing the  anchor-texts

**Performing these steps can be tricky and there are better option [see README](../../README.md#hosting)**

* **Make sure you have alreay followed the [the required steps Steps](../../README.md#hosting) explained in [README.md](../../README.md)**
* **You can find most of the intermediate files in our website reporitory (the permament address can be found [here](../../README.md#api)) **

* Note that all of the paths are hard-coded and you need to modify them, consistently, and carefully!**

### Step 0: Install [Solr](http://lucene.apache.org/solr/)
I assume it is installed in `~/backup/`

### Step 1. Download the Wikipedia

Download the Wikipedia dump to `../../../data/enwiki20160305/original`

wget https://dumps.wikimedia.org/enwiki/20160305/enwiki-20160305-pages-articles.xml.bz2 -P `../../../data/enwiki20160305/original`

### Step 2: Run the following cell to create the `toannonated.py`

In [ ]:
%%writefile toannonated.py
'''Converts the output of WikiExtractor.py to a text file with each line a json object
    Each object has the following fields:
    id: Wikipedia id
    title: The title
    opening_text: The first paragraph of the page
    opening_annotation: The annotation of the oppenint text which is itself another json objec
    text: The text
    annotation": The annotation of the text, which is itself another json object 

'''
import re
import json
import sys
import urllib
import sys
from HTMLParser import HTMLParser
sys.path.insert(0,'..')
from wikisim.wikipedia import *

__author__ = "Armin Sajadi"
__copyright__ = "Copyright 215, The Wikisim Project"
__credits__ = ["Armin Sajadi"]
__license__ = "GPL"
__version__ = "1.0.1"
__maintainer__ = "Armin Sajadi"
__email__ = "sajadi@cs.dal.ca"
__status__ = "Development"


fileinput = sys.stdin

# import StringIO
# fileinput = StringIO.StringIO(inputstr)
def replacelinks(text):
    hp = HTMLParser()
    
    annotations = []
    deltaStringLength = 0
    hrefreg=r'<a href="([^"]+)">([^>]+)</a>'
    ms = re.finditer(hrefreg, text)
    
    
    for m in ms:     

        url = m.group(1)
        # in the parser, url->encode->quote->encode (does nothing, already unicode)->write to file
        # we already have decoded while reading, we have to encode back, unquote, and decode, so that
        # the last encode on writing can make it back to unicode
        url=url.encode('utf-8')
        url =  urllib.unquote(url)
        url = url.decode('utf-8')
        #sometimes we have [[&]] that has been encoded to &amp; 
        #sometimes we have [[&amp]] that has been encoded to &amp;amp; 
        #so we unexcape twice!
        
        url=hp.unescape(url)
        url=hp.unescape(url)
        url=url.replace(u"\xA0"," ")
        
        x = url.find("#")
        if x!=-1:
            url=url[:x]
        #BUG: so what if x==0?  maybe the current approach, empty "dest" is not bad
        antext = m.group(2)
        if '//' not in url:
            annotations.append({
                "url"    :   url, 
                "surface_form" :   antext, 
                "from"  :   m.start() - deltaStringLength,
                "to"    :   m.start() - deltaStringLength+len(antext)
            })

        deltaStringLength += len(m.group(0)) - len(antext)

    #As a second step, replace all links in the article by their label
    text = re.sub(hrefreg, lambda m:m.group(2), text)  
    return annotations, text
def process():
    hp = HTMLParser()
    rstart=r'<doc id="(.*)" url="(.*)" title="(.*)">'
    rend=r'</doc>'

#     open_ann= open ('open_annotation.json', 'w')
#     ann= open ('annotation.json', 'w')
#     wikitext=open('wikitext.json','w')
    state=0
    page=""
    textlist=[]
    while True:
        line = fileinput.readline()
        if not line:
            break
        line = line.decode('utf-8').strip()
        
        if not line:
            continue
        if re.match(rend,line):
            # If you want to check the title, make sure to encode!
            if textlist and (id2title(wid) is not None):
                opening_ann, opening_text = replacelinks(opening_text)
                ann,text = replacelinks("\n".join(textlist))
                
#                 ElasticSearch   
#                 Buggy: convert annotations to text or do something about it
#                 print json.dumps({"index":{"_type":"page","_id":wid}}, ensure_ascii=False).encode('utf-8')
#                 page={"title": wtitle, "opening_text":opening_text, "opening_annotation":opening_ann,
#                       "text":text, "annotation": ann}
                
#                 print json.dumps(page, ensure_ascii=False).encode('utf-8')
                
#               General
                page={"id":wid, "title": wtitle, "opening_text":opening_text, "opening_annotation":opening_ann,
                      "text":text, "annotation": ann}
                
                print json.dumps(page, ensure_ascii=False).encode('utf-8')
                
            textlist=[]    
            state=0
            continue
        if state==0:
            if re.match('<doc', line):
                ms = re.match(rstart, line)
                wid=ms.group(1)
                wtitle=hp.unescape(ms.group(3)).replace(u"\xA0"," ")

                state = 1
            continue    

        
        if state==1:
            state=2
            continue
        textlist.append(line)
            
        if state==2:
            opening_text=line
            state=3
            continue
            
#     wikitext.close()
#     ann.close()
if __name__ == "__main__": process()

# Extract Anchors
### Step 3. Run the following cell to create `extractanchors.py`

In [ ]:
%%writefile extractanchors.py
''' Extract all the anchor text with their target url
'''
import json
import sys
import urllib
import StringIO

__author__ = "Armin Sajadi"
__copyright__ = "Copyright 215, The Wikisim Project"
__credits__ = ["Armin Sajadi"]
__license__ = "GPL"
__version__ = "1.0.1"
__maintainer__ = "Armin Sajadi"
__email__ = "sajadi@cs.dal.ca"
__status__ = "Development"

fileinput = sys.stdin

#fileinput = StringIO.StringIO(inputstr)
while True:
    line = fileinput.readline()
    if not line:
        break
    line = line.decode('utf-8').strip()
    js = json.loads(line)    
    if "annotation" not in js:
        continue
    for a in js["annotation"]:
        print u"{0}\t{1}".format(a["surface_form"],a["url"].replace(" ","_")).encode('utf-8')
                
    

### Step 4. Run  [WikiExtractor](https://github.com/attardi/wikiextractor)
In case you face a problem with the latest one, try the version we are keeping in this directory

`mkdir ../../../data/enwiki20160305/texts`

`nohup ./WikiExtractor.py -o ../../../data/enwiki20160305/texts/xmltexts -b 1000M  -l  --no-templates ../../../data/enwiki20160305/original/enwiki-20160305-pages-articles.xml.bz2 > wikiext.log 2>&1 &`

### Step 5. Annotate The Output
`time cat ../../../data/enwiki20160305/texts/xmltexts/wiki_* | python toannonated.py > ../../../data/enwiki20160305/texts/enwiki-20160305-annonated.json`

### Step 6. Extract the Anchors

`time cat ../../../data/enwiki20160305/texts/enwiki-20160305-annonated.json | python extractanchors.py > ../../../data/enwiki20160305/texts/enwiki-20160305-anchor-titles-from-text`


# Augmenting anchors with titles
There are some titles with no anchors, such as:

* Meiluawati

* Zdeněk_Svoboda

### Step 7. Run the following cell to create `titles2anchors.py`



In [ ]:
%%writefile titles2anchors.py
''' Add the titles to anchor list
'''
import MySQLdb

__author__ = "Armin Sajadi"
__copyright__ = "Copyright 215, The Wikisim Project"
__credits__ = ["Armin Sajadi"]
__license__ = "GPL"
__version__ = "1.0.1"
__maintainer__ = "Armin Sajadi"
__email__ = "sajadi@cs.dal.ca"
__status__ = "Development"

_db = MySQLdb.connect(host="127.0.0.1",port=3307,user='root',passwd="emilios",db="enwiki20160305")
_cursor = _db.cursor()
_cursor.execute("select page_title from page where page_namespace=0");
rows = _cursor.fetchall();
for row in rows:
    title=row[0]
    surface_form = title;
    i=surface_form.find('(')
    if i!=-1:
        surface_form = surface_form[:i]
    surface_form = surface_form.replace('_', ' ')
    surface_form = surface_form.strip()
    
    print surface_form+"\t"+title
    

### Step 8: Adding Titles to Anchors

`time python titles2anchors.py > ../../../data/enwiki20160305/texts/enwiki-20160305-anchor-from-titles.tsv`

`cp ../../../data/enwiki20160305/texts/enwiki-20160305-anchor-titles-from-text.tsv  ../../../data/enwiki20160305/texts/enwiki-20160305-anchor-titles.tmp.tsv`

`cat ../../../data/enwiki20160305/texts/enwiki-20160305-anchor-from-titles.tsv >> ../../../data/enwiki20160305/texts/enwiki-20160305-anchor-titles.tmp.tsv`


### Step 9 . Prepare the anchor-titles file
`time cat ../../../data/enwiki20160305/texts/enwiki-20160305-anchor-titles.tmp.tsv | sort| uniq -c | sed 's/^ *//;s/ /\t/' > ../../../data/enwiki20160305/texts/enwiki-20160305-anchor-titles.tsv`


# Resolving redirects 
We want each anchor to map to a non-redirect page

### Step 10. Run the following cell to create `resolveredirs.py`

In [ ]:
%%writefile resolveredirs.py
'''Incase an anchor is pointing to a redirect page, it replaces the target to the final destination
'''


import sys
sys.path.append('../../cgi-bin/')
from collections import defaultdict
import imp
import json
import urllib
from wikipedia import *
import StringIO

__author__ = "Armin Sajadi"
__copyright__ = "Copyright 215, The Wikisim Project"
__credits__ = ["Armin Sajadi"]
__license__ = "GPL"
__version__ = "1.0.1"
__maintainer__ = "Armin Sajadi"
__email__ = "sajadi@cs.dal.ca"
__status__ = "Development"

fileinput = sys.stdin
errout = sys.stderr
#fileinput = StringIO.StringIO(inputstr)
pairdict = defaultdict(int)
while True:
    line = fileinput.readline()
    if not line:
        break
    line = line.strip()
    tp = line.split("\t")
    if len(tp) !=3:
        errout.write("Line Error: " + line)
        continue
    freq,anchor,title=tp
    wid=title2id(title)
    if wid is None:
        title=title[0].upper()+title[1:]
    wid=title2id(title)
    if wid is None:
        errout.write(anchor+"\t"+title+"\t"+str(freq)+"\n")
        continue
    pairdict[(anchor,wid)] += int(freq)
for (anchor,wid),freq in pairdict.iteritems():
    print anchor+"\t"+str(wid)+"\t"+str(freq)

### Step 11. Applying the resolver

`time cat ../../../data/enwiki20160305/texts/enwiki-20160305-anchor-titles.tsv | python resolveredirs.py > ../../../data/enwiki20160305/texts/enwiki-20160305-anchor-ids.tsv 2> ../../../data/enwiki20160305/texts/enwiki-20160305-anchor-ids.dead.tsv`

### Step 12. Importing to mysql
`cd mysqltables`

`bash importall ../../../../data/enwiki20160305/texts 20160305 root emilios`

`cd ..`

### Step 13. Extracting anchors lists
`cat ../../../data/enwiki20160305/texts/enwiki-20160305-anchor-ids.tsv| cut -f1 -d$'\t'| sort | uniq > ../../../data/enwiki20160305/texts/enwiki-20160305-anchors.tsv`



# ToSolr v2. converts the annotations to a jsonstring
The following script converts our annotated Wikipedia to something Solr can accept

It simply encode the json objects containing the annotations to a string, to stop solr from importing them

### Step 14. Run the following cell to create `tosolr.py`

In [8]:
%%writefile solrsettings/tosolr.py
import json
import sys
import os
import StringIO

__author__ = "Armin Sajadi"
__copyright__ = "Copyright 215, The Wikisim Project"
__credits__ = ["Armin Sajadi"]
__license__ = "GPL"
__version__ = "1.0.1"
__maintainer__ = "Armin Sajadi"
__email__ = "sajadi@cs.dal.ca"
__status__ = "Development"


fileinput = sys.stdin

#fileinput = StringIO.StringIO(inputstr)

n=1000
count=0;
f=None
os.mkdir("chunks")
seperator=''
firstline=None
while True:
    line = fileinput.readline()
    if not line:
        break
        
    line = line.decode('utf-8').strip()
    if not line:
        continue
        
    if count%n==0:
        if f is not None:
            f.write('\n]\n')
            f.close()
        f=open('chunks/chunk'+str(int(count/n)).zfill(10)+'.json','w')    
        f.write('[\n')
        firstline=True
        seperator=""
    count +=1
    
    page=json.loads(line)
    page["annotation"]=json.dumps(page["annotation"], ensure_ascii=False)
    page["opening_annotation"]=json.dumps(page["opening_annotation"], ensure_ascii=False)
    f.write(seperator+json.dumps(page, ensure_ascii=False).encode('utf-8'))
    if firstline:
        firstline=False
        seperator=",\n"
    
    
if f is not None:
    f.write('\n]\n')
    f.close()
    

    
    

Writing solrsettings/tosolr.py


### Step 15. Preparing solr importable
`cd solrsettings`

`time python tosolr.py <../../../../data/enwiki20160305/texts/enwiki-20160305-annonated.json`

### Step 16. creating solr nodes

`solr create -c enwiki20160305`

`solr stop`

`cp conf/* ~/backup/solr-6.0.0/server/solr/enwiki20160305/conf/`

`solr start`

### Step17. importing to solr

`nohup time bash loadwiki >load.txt 2>&1 &`

`cd ..` 

**Check the count**

curl 'http://localhost:8983/solr/enwiki20160305/select?indent=on&q=*:*&rows=0&wt=json'

# [SolrTextTagger](https://github.com/OpenSextant/SolrTextTagger) preparation

### Step 18. Prepare SolrTextTagger

* `bin/solr create -c enwikianchors20160305`

* `bin/solr stop`

* `cp -r ~/backup/projects/wikisim/preparation_script/text/solrtagger/lib/ server/solr/`

* `cp ~/backup/projects/wikifier/preparation_script/solrtagger/conf/* server/solr/enwikianchors20160305/conf/`

* `curl -X POST --data-binary @enwiki-20160305-anchors.tsv.txt -H 'Content-type:application/csv' \
  'http://localhost:8983/solr/enwikianchors20160305/update?commit=true&optimize=true&separator=%09&encapsulator=%00&fieldnames=name'`
  
  


# Sample code to check Random Access to paragraphs with links
### You can safely ignore this

In [ ]:
import re
import random
import requests
import json 
qstr = 'http://localhost:8983/solr/enwiki20160305/select'
q='title:Bruce_Lee'
params={'indent':'on', 'wt':'json', 'q':q}
r = requests.get(qstr, params=params)
r =  r.json()['response']['docs'][0]

pos=[m.start() for m in re.finditer('\n', r['text'])]    
n = index = random.randint(0,len(pos)-1)
s = pos[n]
e = pos[n+1] if n < len(pos)-1 else len(r['text'])
par = r['text'][s:e]
print r['title']
print "par no:",n, par[:10],"...", par[len(par)-10:]
for ann in json.loads(r['annotation']):
    #print ann
    if int(ann['from']) < s: 
        continue
    if int(ann['to']) >= e: 
        break
    surfintext = r['text'][ann['from']:ann['to']]
    print "detected: %s, in annotation: %s, title: %s" % (surfintext, ann[u'surface_form'], ann['url'])
        

# Creating the context core

We will create a solr core that indexes the context (a window of size 20) for any anchor text  

We will use this core for disambiguation. The implementation is multi-threaded to make the process as fast as possible.

# Step 19. Run the following script to create `entitycontext_th.py`

In [10]:
%%writefile entitycontext_th.py 

''' Generate a json file (contexts.json) where eahc line is an anchor text 
    with its left and right context
'''

import os
import requests
import json
import string
import time
import sys
from multiprocessing import Pool, Process, Manager 
import functools
sys.path.insert(0,'../..')

from memapi.memwiki import *

startTime = time.time()
load_tables()
print 'wiki loaded to memory'    
print time.time()-startTime
sys.stdout.flush()

batch_rows=30000
init_start=0
process_no=30
max_rows=-1
window_size=10

qstr = 'http://localhost:8983/solr/enwiki20160305/select'
q='*:*'
#q='id:"60508"'

params={'indent':'on', 'wt':'json', 'q':q, "start":init_start,
        "rows":batch_rows}
home = os.path.expanduser("~");
#titles_out.write('kalam\n')

start=0
max_count=0
count_nottop=0

def find_nth(text, c, i, n, direction=0):
    if direction ==0:
        offset = text.find(c, i+1)
    else:    
        offset = text.rfind(c, 0,i)
    while offset >= 0 and n > 1:
        if direction ==0:
            offset = text.find(c, offset+len(c))
        else:    
            offset = text.rfind(c, 0, offset)
        n -= 1
    return offset

        
def getsentence(r, ann):
    text = r["text"]
    source_entity=r['id']
    source_title=r['title']
    
    entity = ann['url'] if ann['url']!="" else r['title']
    entityid = title2id(entity)
    if entityid is None:
        entity=entity[0].upper()+entity[1:]    
        entityid = title2id(entity)
    if entityid is None:
        return None
    trans_tab = dict.fromkeys(map(ord, string.punctuation), None)
    
    s = find_nth(text,' ', ann['from'],2*window_size,1)

    if s==-1:
        s=0
    l = text[s:ann['from']].translate(trans_tab).strip().split()
    l=" ".join(l[-window_size:])

    s = find_nth(text,' ', ann['from'],2*window_size,0)
    if s==-1:
        s=len(text)
    r = text[ann['to']:].translate(trans_tab).strip().split()
    r=" ".join(r[:window_size])
    pno = text[:ann['from']].count('\n')
    return {"left":l, "anchor":ann['surface_form'],"source_entity":source_entity,"source_title":source_title,
            "entity": entity, "entityid":entityid, "right": r, "paragraph_no":pno}


def process(r, tq, cq):
    tq.put(r['title'].encode('utf-8'))
    annotations = json.loads(r['annotation'])    
    for ann in annotations:
        sentence = getsentence(r, ann)
        if sentence is None:
            continue
        entity_context = json.dumps(sentence, ensure_ascii=False).encode('utf-8')
        cq.put(entity_context)
        
pool =Pool(process_no) 

def worker(fname, q):
    w = open(fname,'w')
    print "Writer started"
    sys.stdout.flush()
    while True:
        s = q.get()
        if s=='kill':
            print "Writer worker closing"
            sys.stdout.flush()
            break
        w.write(s+"\n")
    w.close()    
    
manager= Manager()

titles_name = os.path.join(home,'backup/data/enwiki20160305/cmod/titles.txt')
title_q = manager.Queue()

cont_name = os.path.join(home,'backup/data/enwiki20160305/cmod/contexts.json')
cont_q = manager.Queue()


title_proc = Process(target=worker, args=(titles_name, title_q))
title_proc.start()   
        
cont_proc = Process(target=worker, args=(cont_name, cont_q))
cont_proc.start()   


start=init_start
rows=0
while True:
    params["start"] = start

    r = requests.get(qstr, params=params)
    print ("A batch retrieved: "+ str(start)+'\n')
    sys.stdout.flush()
    if len(r.json()['response']['docs'])==0:
        break
    #print r.json()['response']['docs']
    D = r.json()['response']['docs']
        
    pool.map(functools.partial(process, tq=title_q, cq=cont_q ), D)
    #pool.close() 
    #pool.join() 
    
    start += batch_rows
    rows += batch_rows
    if max_rows !=-1 and rows >= max_rows:
        break
        
cont_q.put('kill')    
title_q.put('kill')

title_proc.join()
cont_proc.join()

print 'done'    

print time.time()-startTime

Writing entitycontext_th.py


### Step 20. Run the script to create `contexts.json`

`mkdir ../../../data/enwiki20160305/cmod`

`python entitycontext_th.py`



### Step 21. Split the file
`cd solrcontexts`

`mkdir chunks`

`cd chunks`

`cat ../../../../../data/enwiki20160305/cmod/contexts.json | split -a 10 -l 500 - basename`

`cd ..`

### Step 22. Preparing solr importable
`bash tosolr.sh`

### Step 23. creating solr nodes


`solr create -c enwiki20160305_context`

`solr stop`

`cp conf/* ../../../wikipedia/solr-6.0.0/server/solr/enwiki20160305_context/conf/`

`solr start`


### Step 24. importing to solr
`nohup time bash loadwiki >load.txt 2>&1 &`

`cd ..

**Check the count**

curl 'http://localhost:8983/solr/enwiki20160305_context/select?indent=on&q=*:*&rows=0&wt=json'